In [1]:
import pandas as pd     
import numpy as np
import nltk
import string
import nltk
import matplotlib.pyplot as plt
import re
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
data=pd.read_csv("Coursera.csv")

In [3]:
data.head()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Michigan State University,Beginner,4.8,https://www.coursera.org/learn/write-a-feature...,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Coursera Project Network,Beginner,4.8,https://www.coursera.org/learn/canvas-analysis...,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,�cole Polytechnique,Advanced,4.1,https://www.coursera.org/learn/silicon-thin-fi...,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,IESE Business School,Intermediate,4.8,https://www.coursera.org/learn/operational-fin...,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Coursera Project Network,Beginner,4.6,https://www.coursera.org/learn/single-table-sq...,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [4]:
data.tail()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
3517,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,Beginner,4.6,https://www.coursera.org/learn/python-data-vis...,"In the capstone, students will build a series ...",Databases syntax analysis web Data Visuali...
3518,Patrick Henry: Forgotten Founder,University of Virginia,Intermediate,4.9,https://www.coursera.org/learn/henry,"�Give me liberty, or give me death:� Rememberi...",retirement Causality career history of the ...
3519,Business intelligence and data analytics: Gene...,Macquarie University,Advanced,4.6,https://www.coursera.org/learn/business-intell...,�Megatrends� heavily influence today�s organis...,analytics tableau software Business Intellig...
3520,Rigid Body Dynamics,Korea Advanced Institute of Science and Techno...,Beginner,4.6,https://www.coursera.org/learn/rigid-body-dyna...,"This course teaches dynamics, one of the basic...",Angular Mechanical Design fluid mechanics F...
3521,Architecting with Google Kubernetes Engine: Pr...,Google Cloud,Intermediate,4.7,https://www.coursera.org/learn/deploying-secur...,"In this course, ""Architecting with Google Kube...",bigtable bigquery SQL Google Cloud Platform...


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3522 entries, 0 to 3521
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Course Name         3522 non-null   object
 1   University          3522 non-null   object
 2   Difficulty Level    3522 non-null   object
 3   Course Rating       3522 non-null   object
 4   Course URL          3522 non-null   object
 5   Course Description  3522 non-null   object
 6   Skills              3522 non-null   object
dtypes: object(7)
memory usage: 192.7+ KB


In [6]:
data.describe()

,Course Name,University,Difficulty Level,Course Rating,Course URL,Course Description,Skills
count,3522,3522,3522,3522,3522,3522,3522
unique,3416,184,5,31,3424,3397,3424
top,Google Cloud Platform Fundamentals: Core Infra...,Coursera Project Network,Beginner,4.7,https://www.coursera.org/learn/gcp-fundamentals,This course introduces you to important concep...,Google Cloud Platform Big Data Cloud Infrast...
freq,8,562,1444,740,8,8,8


In [7]:
data.columns

Index(['Course Name', 'University', 'Difficulty Level', 'Course Rating',
       'Course URL', 'Course Description', 'Skills'],
      dtype='object')

In [8]:
data.shape 

(3522, 7)

In [9]:
data.isnull().sum()

Course Name           0
University            0
Difficulty Level      0
Course Rating         0
Course URL            0
Course Description    0
Skills                0
dtype: int64

In [10]:
data['Difficulty Level'].value_counts()


Difficulty Level
Beginner          1444
Advanced          1005
Intermediate       837
Conversant         186
Not Calibrated      50
Name: count, dtype: int64

In [11]:
data= data[['Course Name','Difficulty Level','Course Description','Skills']]
data.head(5)


,Course Name,Difficulty Level,Course Description,Skills
0,Write A Feature Length Screenplay For Film Or ...,Beginner,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...
1,Business Strategy: Business Model Canvas Analy...,Beginner,"By the end of this guided project, you will be...",Finance business plan persona (user experien...
2,Silicon Thin Film Solar Cells,Advanced,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...
3,Finance for Managers,Intermediate,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...
4,Retrieve Data using Single-Table SQL Queries,Beginner,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...


In [12]:
#Tags Column The tags column is the combination of the following columns : Course Name + Difficulty Level + Course Description + Skills
data['tags'] = data['Course Name'] + data['Difficulty Level'] + data['Course Description'] + data['Skills']
data.head(5)

,Course Name,Difficulty Level,Course Description,Skills,tags
0,Write A Feature Length Screenplay For Film Or ...,Beginner,Write a Full Length Feature Film Script In th...,Drama Comedy peering screenwriting film D...,Write A Feature Length Screenplay For Film Or ...
1,Business Strategy: Business Model Canvas Analy...,Beginner,"By the end of this guided project, you will be...",Finance business plan persona (user experien...,Business Strategy: Business Model Canvas Analy...
2,Silicon Thin Film Solar Cells,Advanced,This course consists of a general presentation...,chemistry physics Solar Energy film lambda...,Silicon Thin Film Solar CellsAdvancedThis cour...
3,Finance for Managers,Intermediate,"When it comes to numbers, there is always more...",accounts receivable dupont analysis analysis...,Finance for ManagersIntermediateWhen it comes ...
4,Retrieve Data using Single-Table SQL Queries,Beginner,In this course you�ll learn how to effectively...,Data Analysis select (sql) database manageme...,Retrieve Data using Single-Table SQL QueriesBe...


In [13]:
data = data[['Course Name','tags']]
data.head()


,Course Name,tags
0,Write A Feature Length Screenplay For Film Or ...,Write A Feature Length Screenplay For Film Or ...
1,Business Strategy: Business Model Canvas Analy...,Business Strategy: Business Model Canvas Analy...
2,Silicon Thin Film Solar Cells,Silicon Thin Film Solar CellsAdvancedThis cour...
3,Finance for Managers,Finance for ManagersIntermediateWhen it comes ...
4,Retrieve Data using Single-Table SQL Queries,Retrieve Data using Single-Table SQL QueriesBe...


In [14]:
#Data Pre-Processing
data['Course Name'] = data['Course Name'].astype(str)
data['tags'] = data['tags'].astype(str)
data['tags'] = data['tags'].str.lower()


In [15]:
#Data_preproccing
data['Course Name'] = data['Course Name'].str.replace('&', '', regex=False)
data['Course Name'] = data['Course Name'].str.replace(':', '', regex=False)
data['Course Name'] = data['Course Name'].str.replace('_', '', regex=False)
data['Course Name'] = data['Course Name'].str.replace('-', '', regex=False)
data['Course Name'] = data['Course Name'].str.replace('(', '', regex=False)
data['Course Name'] = data['Course Name'].str.replace(')', '', regex=False)
data['Course Name'] = data['Course Name'].str.replace('+', '', regex=False)
data['Course Name'] = data['Course Name'].str.replace(',,', ',', regex=False)
data['Course Name'] = data['Course Name'].str.replace(',,,', ',', regex=False)
data['Course Name'] = data['Course Name'].str.replace(' ', ',', regex=False)
data['Course Name'] = data['Course Name'].str.replace("'", '', regex=False)
data['Course Name'] = data['Course Name'].str.replace(',,', ',', regex=False)

data['tags'] = data['tags'].str.replace('&', '', regex=False)
data['tags'] = data['tags'].str.replace(':', '', regex=False)
data['tags'] = data['tags'].str.replace('_', '', regex=False)
data['tags'] = data['tags'].str.replace('-', '', regex=False)
data['tags'] = data['tags'].str.replace('(', '', regex=False)
data['tags'] = data['tags'].str.replace(')', '', regex=False)
data['tags'] = data['tags'].str.replace('+', '', regex=False)
data['tags'] = data['tags'].str.replace(',,', ',', regex=False)
data['tags'] = data['tags'].str.replace(',,,', ',', regex=False)
data['tags'] = data['tags'].str.replace(' ', ',', regex=False)
data['tags'] = data['tags'].str.replace("'", '', regex=False)
data['tags'] = data['tags'].str.replace(',,', ',', regex=False)


In [16]:
#data-preproccing
data['Course Name'] = data['Course Name'].str.replace(',,', ',', regex=False)
data['Course Name'] = data['Course Name'].str.replace('.', ' ', regex=False)
data['Course Name'] = data['Course Name'].str.replace('�', ',', regex=False)
data['Course Name'] = data['Course Name'].str.replace(',', ' ', regex=False)
data['tags'] = data['tags'].str.replace('.', ' ', regex=False)
data['tags'] = data['tags'].str.replace(',,', ',', regex=False)
data['tags'] = data['tags'].str.replace('�', ',', regex=False)
data['tags'] = data['tags'].str.replace(',', ' ', regex=False)
data['tags'] = data['tags'].str.replace('  ', ' ', regex=False)


In [17]:
data['tags'].iloc[1]

'business strategy business model canvas analysis with mirobeginnerby the end of this guided project you will be fluent in identifying and creating business model canvas solutions based on previous highlevel analyses and research data this will enable you to identify and map the elements required for new products and services furthermore it is essential for generating positive results for your business venture this guided project is designed to engage and harness your visionary and exploratory abilities you will use proven models in strategy and product development with the miro platform to explore and analyse your business propositions we will practice critically examining results from previous analysis and research results in deriving the values for each of the business model sections finance business plan persona user experience business model canvas planning business project product development presentation strategy business businessstrategy'

In [18]:
#stopword like : is are ..
nltk.download("stopwords")
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saidi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
#removing stopwords
def remove_stopwords(text):
    return " ".join([word for word in text.split() if word.lower() not in stop_words])
data['cleaned_tags'] = data['tags'].apply(remove_stopwords)


In [20]:
#lemmaization ex: running=run .
import spacy
nlp = spacy.load("en_core_web_sm")


In [21]:
def lemmatize_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])
data['lemmatized_tags'] = data['tags'].apply(lemmatize_text)


###   CountVectorizer

In [22]:
#CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer( max_features=5000)
vectors = cv.fit_transform(data['lemmatized_tags']).toarray()

In [23]:
data.rename(columns = {'Course Name':'course_name'}, inplace = True)

In [24]:
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text


In [25]:
def preprocess_input(course_name, vectorizer):
    course_name = preprocess_text(course_name)  
    course_name_lemmatized = lemmatize_text(course_name)  
    course_vector = vectorizer.transform([course_name_lemmatized]).toarray()  
    return course_vector

In [26]:
def recommend(course_name, df, vectorizer, vectors, top_n=10):
    course_vector = preprocess_input(course_name, vectorizer)
    
    similarity_scores = cosine_similarity(course_vector, vectors)
    similarity_scores = similarity_scores[0]  
    
    similar_courses = [(index, score) for index, score in enumerate(similarity_scores)]
    similar_courses_sorted = sorted(similar_courses, key=lambda x: x[1], reverse=True)
    
    print(f"Top {top_n} courses similar to '{course_name}':")
    for i in similar_courses_sorted[:top_n]:
        print(df.iloc[i[0]]['course_name'])  

recommend("Introduction to Machine Learning", data, cv, vectors, top_n=10)

Top 10 courses similar to 'Introduction to Machine Learning':
Introduction to Applied Machine Learning
Introduction to TensorFlow for Artificial Intelligence Machine Learning and Deep Learning
Introduction to Machine Learning
Machine Learning With Big Data
Machine Learning for Data Analysis
Machine Learning Foundations A Case Study Approach
Machine Learning with H2O Flow
Optimizing Machine Learning Performance
Machine Learning Algorithms Supervised Learning Tip to Tail
Machine Learning Using SAS Viya


 ###   using TF-IDF Vectorizer


In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

In [28]:
vectors = tfidf_vectorizer.fit_transform(data['tags'])

In [29]:
# Define parameters for Q-Learning
num_courses = len(data)
q_table = np.zeros((num_courses, num_courses))  # Q-Table: (state x action)
learning_rate = 0.1
discount_factor = 0.9
epsilon = 0.2  # For exploration vs. exploitation

In [30]:
similarity_matrix = cosine_similarity(vectors)

In [31]:
# Normalize similarity values for rewards
rewards = similarity_matrix / np.max(similarity_matrix)

In [32]:
# Q-Learning Algorithm
import random
def q_learning(num_episodes=1000):
    for episode in range(num_episodes):
        # Start with a random course
        state = random.randint(0, num_courses - 1)
        done = False
        while not done:
            # Choose action using epsilon-greedy policy
            if random.uniform(0, 1) < epsilon:
                action = random.randint(0, num_courses - 1)  # Explore
            else:
                action = np.argmax(q_table[state])  # Exploit
            
            # Get reward and update Q-value
            reward = rewards[state, action]
            next_state = action
            max_next_q = np.max(q_table[next_state])
            
            # Update Q-Value using the Q-Learning formula
            q_table[state, action] += learning_rate * (reward + discount_factor * max_next_q - q_table[state, action])
            
            # Transition to the next state
            state = next_state
            
            # End if reward is very low (no further improvement)
            if reward < 0.1:
                done = True

In [33]:
# Train the Q-Learning agent
q_learning(num_episodes=5000)

In [34]:
def recommend(course_name, df, tfidf_vectorizer, top_n=10):
    course_vector = tfidf_vectorizer.transform([course_name])
    similarity_scores = cosine_similarity(course_vector, vectors)
    similarity_scores = similarity_scores[0]
    similar_courses = [(index, score) for index, score in enumerate(similarity_scores)]
    similar_courses_sorted = sorted(similar_courses, key=lambda x: x[1], reverse=True)
    print(f"Top {top_n} courses similar to '{course_name}':")
    for i in similar_courses_sorted[:top_n]:
        print(df.iloc[i[0]]['course_name'])  
    
recommend("Introduction to Machine Learning", data, tfidf_vectorizer, top_n=10)


Top 10 courses similar to 'Introduction to Machine Learning':
Introduction to Applied Machine Learning
Optimizing Machine Learning Performance
Machine Learning for Data Analysis
Machine Learning Foundations A Case Study Approach
Machine Learning With Big Data
Developing AI Applications on Azure
Machine Learning for All
How Google does Machine Learning
Introduction to TensorFlow for Artificial Intelligence Machine Learning and Deep Learning
Machine Learning with H2O Flow


In [35]:
recommend("Machine Learning", data, tfidf_vectorizer, top_n=10)

Top 10 courses similar to 'Machine Learning':
Introduction to Applied Machine Learning
Machine Learning for Data Analysis
Developing AI Applications on Azure
Optimizing Machine Learning Performance
Machine Learning Foundations A Case Study Approach
How Google does Machine Learning
Machine Learning for All
Machine Learning With Big Data
Machine Learning with Python
Machine Learning with Python


In [36]:
recommend("Python for Data Science", data, tfidf_vectorizer, top_n=10)

Top 10 courses similar to 'Python for Data Science':
Python Data Analysis
Python Data Representations
Python for Data Science and AI
Python for Data Science and AI
Python for Data Science and AI
Python for Data Science and AI
Introduction to Data Science in Python
Python Data Structures
Python Data Visualization
Python Data Structures


In [37]:
#save the model
import pickle
with open("tfidf_vectorizer.pkl", "wb") as file:
    pickle.dump(tfidf_vectorizer, file)

In [38]:
with open("q_table.pkl", "wb") as file:
    pickle.dump(q_table, file)

In [42]:
tfidf_vectorizer = pickle.load(open("tfidf_vectorizer.pkl", "rb"))
q_table = pickle.load(open("q_table.pkl", "rb"))
data = pd.read_csv("Coursera.csv")
data['tags'] = data['Course Name'] + data['Difficulty Level'] + data['Course Description'] + data['Skills']

vectors = tfidf_vectorizer.transform(data['tags'])
#probability of the course recommendations
def recommend(course_name, df, tfidf_vectorizer, top_n=10):
    course_vector = tfidf_vectorizer.transform([course_name])
    similarity_scores = cosine_similarity(course_vector, vectors)
    similarity_scores = similarity_scores[0]
    similar_courses = [(index, score) for index, score in enumerate(similarity_scores)]
    similar_courses_sorted = sorted(similar_courses, key=lambda x: x[1], reverse=True)
    print(f"Top {top_n} courses similar to '{course_name}':")
    for i in similar_courses_sorted[:top_n]:
        print(df.iloc[i[0]]['Course Name'], i[1])
    
recommend("Introduction to Machine Learning", data, tfidf_vectorizer, top_n=10)

Top 10 courses similar to 'Introduction to Machine Learning':
Introduction to Applied Machine Learning 0.6618582820250818
Machine Learning for Data Analysis 0.4984799492527886
Optimizing Machine Learning Performance 0.4958968369739559
Machine Learning With Big Data 0.47569393139314425
Machine Learning Foundations: A Case Study Approach 0.47520920890741236
How Google does Machine Learning 0.47155555042382336
Machine Learning for All 0.44639249457453933
Developing AI Applications on Azure 0.4396147620726836
Machine Learning with Python 0.4357634804534163
Machine Learning with Python 0.4357634804534163


In [43]:
#recommendation of the course only top 3 and also display the skills of the course below the course name
data = pd.read_csv("Coursera.csv")
def recommend(course_name, df, tfidf_vectorizer, top_n=3):
    course_vector = tfidf_vectorizer.transform([course_name])
    similarity_scores = cosine_similarity(course_vector, vectors)
    similarity_scores = similarity_scores[0]
    similar_courses = [(index, score) for index, score in enumerate(similarity_scores)]
    similar_courses_sorted = sorted(similar_courses, key=lambda x: x[1], reverse=True)
    print(f"Top {top_n} courses similar to '{course_name}':")
    for i in similar_courses_sorted[:top_n]:
        print(df.iloc[i[0]]['Course Name'], i[1])
        print(data.iloc[i[0]]['Skills'])
    
recommend("Introduction to Machine Learning", data, tfidf_vectorizer, top_n=3)


Top 3 courses similar to 'Introduction to Machine Learning':
Introduction to Applied Machine Learning 0.6618582820250818
Algorithms  Machine Learning Algorithms  Applied Machine Learning  Test Set  Machine Learning  ordered pair  Training  Human Learning  Deep Learning  business requirements data-science machine-learning
Machine Learning for Data Analysis 0.4984799492527886
Algorithms  Data Analysis  numbers (spreadsheet)  evaluation  Machine Learning  Regression  Dependent And Independent Variables  Human Learning  Regression Analysis  decision tree data-science machine-learning
Optimizing Machine Learning Performance 0.4958968369739559
project  Deep Learning  Strategy  mathematical optimization  Communication  performance  Machine Learning  use case  business requirements  process data-science machine-learning
